In [1]:
import os

In [3]:
%pwd

'/Users/priyaldharmawat/Desktop/CancerClassifier/Cancerclassifier/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/priyaldharmawat/Desktop/CancerClassifier/Cancerclassifier'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from Cancerclassifier.constants import *
from Cancerclassifier.utils.common import read_yaml, create_directories

python(40266) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Note: you may need to restart the kernel to use updated packages.


In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [13]:
import os
import urllib.request as request
import zipfile
from Cancerclassifier import logger
from Cancerclassifier.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-18 19:32:20,677: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-18 19:32:20,687: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-18 19:32:20,693: INFO: common: created directory at: artifacts]
[2025-03-18 19:32:20,693: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-18 19:32:22,107: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 3448693
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "6485116cd1bf46ed2257f869bd8fc08a2b5286cb421c823a071837cb3db4edfe"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0FAC:2F2A16:5E889D:74B17F:67DA0285
Accept-Ranges: bytes
Date: Tue, 18 Mar 2025 23:32:21 GMT
Via: 1.1 varnish
X-Served-By: cache-yyz4563-YYZ
X-Cache: MI